In [ ]:
using Random

pwd()

In [2]:
struct Instance
    name::String
    graphe::BitMatrix # Matrice d'adjacence du graphe
    k::Int
end

mutable struct Solution
    nodecolors::Vector{Int} # coloration
    conflictmatrix::Matrix{Int} # Matrice de conflits si l'on assigne la couleur i au nœud j
    tabuexpiry::Matrix{Int} # Matrice des itérations minimum où une certaine modification est autorisée
    obj::Int # Nombre de conflits
end

Solution(instance::Instance,nodecolors::Vector{Int}) = Solution(
    nodecolors,
    zeros(Int,(instance.k,length(instance))),
    zeros(Int,(instance.k,length(instance))),
    length(instance)^2
)

"""Renvoie le nombre de sommets du graphe de l'instance."""
function Base.length(instance::Instance)
    return size(instance.graphe)[1]
end

function Base.length(solution::Solution)
    return length(solution.nodecolors)
end

In [3]:
function recup_numbre_dans_string(str::String,i::Int) #cette fonction recupere le nombre qui commence à l'indice i dans le string str
    j = i
    while length(str) >= j+1 && str[j+1] >= '0' && str[j+1] <= '9' #verifie si on manipule bien un chiffre
        j+=1
    end
    return (parse(Int64,str[i:j]),j)
end

"""Construit une instance de k-coloration à partir du fichier."""
function read_instance(path::String,k::Int)
    io = open(path)
    b = true
    graphe = BitMatrix
    while (b)
        ligne = readline(io)
        if length(ligne) >= 1 && ligne[1] == 'p'
            n = recup_numbre_dans_string(ligne,8)[1] #nombre de sommets dans le graphe
            graphe = falses(n,n)
        end
        if length(ligne) >= 1 && ligne[1] == 'e'
            u,j = recup_numbre_dans_string(ligne,3)
            v = recup_numbre_dans_string(ligne,j+1)[1]
            graphe[u,v] = true
            graphe[v,u] = true
        end
        if length(ligne) < 1
            b = false
        end
    end
    name = split(path,"/")[2]
    return Instance(name,graphe,k)
end

read_instance

In [4]:
# Définit les instances à évaluer
const instance_list = (
    read_instance("graphs/flat300_26_0.col", 26),
    read_instance("graphs/le450_15c.col", 15),
    read_instance("graphs/dsjc125.1.col", 5),
    read_instance("graphs/dsjc125.9.col", 44),
    read_instance("graphs/dsjc250.1.col", 8),
    read_instance("graphs/dsjc250.9.col", 72),
    read_instance("graphs/dsjc250.5.col", 28),
    read_instance("graphs/dsjc1000.5.col", 86),
    read_instance("graphs/dsjc1000.5.col", 85),
    read_instance("graphs/dsjc1000.5.col", 84)
)

(Instance("flat300_26_0.col", Bool[0 1 … 0 0; 1 0 … 1 1; … ; 0 1 … 0 1; 0 1 … 1 0], 26), Instance("le450_15c.col", Bool[0 0 … 0 0; 0 0 … 1 0; … ; 0 1 … 0 0; 0 0 … 0 0], 15), Instance("dsjc125.1.col", Bool[0 0 … 0 0; 0 0 … 0 0; … ; 0 0 … 0 0; 0 0 … 0 0], 5), Instance("dsjc125.9.col", Bool[0 1 … 1 1; 1 0 … 0 1; … ; 1 0 … 0 0; 1 1 … 0 0], 44), Instance("dsjc250.1.col", Bool[0 0 … 0 0; 0 0 … 1 0; … ; 0 1 … 0 1; 0 0 … 1 0], 8), Instance("dsjc250.9.col", Bool[0 1 … 1 1; 1 0 … 0 1; … ; 1 0 … 0 1; 1 1 … 1 0], 72), Instance("dsjc250.5.col", Bool[0 1 … 1 0; 1 0 … 1 0; … ; 1 1 … 0 0; 0 0 … 0 0], 28), Instance("dsjc1000.5.col", Bool[0 0 … 0 0; 0 0 … 1 1; … ; 0 1 … 0 0; 0 1 … 0 0], 86), Instance("dsjc1000.5.col", Bool[0 0 … 0 0; 0 0 … 1 1; … ; 0 1 … 0 0; 0 1 … 0 0], 85), Instance("dsjc1000.5.col", Bool[0 0 … 0 0; 0 0 … 1 1; … ; 0 1 … 0 0; 0 1 … 0 0], 84))

In [5]:
"""Calcule le nombre de collisions de la coloration."""
function nbr_collision(instance::Instance,solution::Solution)
    compteur = 0 
    for j = 1:length(instance)
        for i = j+1:length(instance)
            compteur += instance.graphe[i,j] && solution.nodecolors[i] == solution.nodecolors[j]
        end
    end
    solution.obj = compteur
    return compteur
end

nbr_collision

In [6]:
"""
Calcule le nombre de collisions au sommet position en lui attribuant couleur.
"""
function nb_collision_sommet_couleur(instance::Instance,solution::Solution,position::Int,couleur::Int)
    n_collision = 0
    for  i = 1:length(solution)
        n_collision += instance.graphe[i,position] && solution.nodecolors[i] == couleur
    end
    return n_collision
end

nb_collision_sommet_couleur

In [7]:
"""Calcule l'ordre dans lequel traiter les sommets dans l'heuristique gloutonne (degré décroissant)."""
function calcul_de_ordre_des_sommets(instance::Instance) 
    deg = vec(sum(instance.graphe,dims=1))
    return sortperm(deg,rev=true)
end

"""Détermine la meilleure couleur à attribuer au nœud."""
function meilleure_couleur_locale(instance::Instance,solution::Solution,position::Int)
    best_c = 1
    best_nb_collision = length(solution)
    for c = 2:instance.k # Les couleurs sont entre 1 et k
        nb_collision = nb_collision_sommet_couleur(instance,solution,position,c)
        if nb_collision < best_nb_collision
            best_c = c
            best_nb_collision = nb_collision
        end
    end
    return best_c
end

meilleure_couleur_locale

# 1) Heuristique gloutonne

L'algorithme glouton choisi:
 - assigne successivement une couleur à chaque nœud
 - les nœuds sont traités dans l'ordre de degré décroissant
 - en choisissant pour chaque nœud la couleur minimisant le nombre de conflits avec ses voisins.

In [8]:
"""Heuristique gloutonne générant une solution par coloration successive des sommets."""
function glouton(instance::Instance)
    solution = Solution(instance,zeros(Int,length(instance))) # la couleur 0 veut dire non colorée
    ordre_des_sommets = calcul_de_ordre_des_sommets(instance)
    for i ∈ ordre_des_sommets
        c = meilleure_couleur_locale(instance,solution,i)
        solution.nodecolors[i] = c
    end
    return solution
end

glouton

In [9]:
"""Génère une solution aléatoire."""
function sol_alea(instance::Instance)
    return Solution(instance,rand(1:instance.k,length(instance)))
end

sol_alea

In [10]:
begin
    instance = instance_list[3]
    aleatoire = sol_alea(instance)

    println(instance.name,"\tk=",instance.k)
    println("SOLUTION ALÉATOIRE:")
    println(aleatoire.nodecolors)
    print("collisions: ")
    println(nbr_collision(instance,aleatoire))
    gloutonne = glouton(instance)
    println("\nSOLUTION GLOUTONNE:")
    println(gloutonne.nodecolors)
    print("collisions: ")
    println(nbr_collision(instance,gloutonne))
end

dsjc125.1.col	k=5
SOLUTION ALÉATOIRE:


[3, 2, 3, 2, 2, 2, 3, 5, 2, 4, 3, 4, 5, 4, 3, 2, 2, 4, 5, 3, 4, 4, 2, 4, 2, 4, 1, 3, 1, 4, 1, 2, 3, 1, 5, 1, 2, 4, 4, 2, 1, 4, 1, 3, 5, 4, 1, 4, 1, 5, 4, 4, 4, 1, 1, 5, 4, 2, 4, 2, 4, 1, 5, 2, 3, 3, 3, 4, 2, 5, 3, 2, 3, 1, 5, 1, 2, 3, 3, 3, 2, 2, 1, 5, 2, 5, 3, 2, 1, 1, 4, 1, 1, 2, 2, 3, 4, 2, 2, 1, 1, 4, 4, 5, 3, 4, 5, 1, 4, 3, 5, 2, 3, 5, 1, 1, 3, 3, 4, 4, 4, 3, 5, 4, 1]
collisions: 158



SOLUTION GLOUTONNE:
[3, 2, 2, 4, 2, 5, 4, 3, 4, 3, 2, 5, 3, 3, 2, 4, 4, 4, 3, 5, 5, 5, 5, 4, 5, 2, 3, 3, 2, 5, 4, 3, 4, 4, 3, 5, 4, 2, 4, 5, 5, 4, 2, 2, 5, 4, 3, 5, 3, 2, 2, 2, 4, 3, 4, 4, 4, 5, 5, 2, 2, 2, 5, 3, 3, 5, 2, 3, 2, 2, 3, 3, 3, 2, 3, 2, 3, 5, 4, 4, 4, 3, 4, 2, 2, 2, 5, 3, 3, 3, 2, 4, 3, 3, 5, 5, 4, 2, 5, 3, 2, 3, 4, 5, 2, 5, 5, 3, 4, 4, 3, 2, 2, 5, 2, 4, 5, 5, 2, 4, 4, 3, 2, 2, 3]
collisions: 47


## Tests numériques de l'heuristique gloutonne

In [11]:
begin
    nsamples = 10
    println("INSTANCE NAME AND k\tMIN CONFL\tMEAN CONFL\tMAX CONFL\tTOTAL TIME\tTIME BEST SOL\tSOLS PER SECOND")
    for instance ∈ instance_list
        conflicts_samples = Vector{Int}(undef,nsamples)
        time_samples = zeros(nsamples)
        for i = 1:nsamples
            solution = glouton(instance)
            start_time = time_ns()
            conflicts = nbr_collision(instance,solution)
            run_time = time_ns()-start_time
            conflicts_samples[i] = conflicts
            time_samples[i] = run_time*1e-9
        end
        println(
            instance.name," k=",instance.k,"\t",
            minimum(conflicts_samples),"\t",
            sum(conflicts_samples)/nsamples,"\t",
            maximum(conflicts_samples),"\t",
            sum(time_samples),"\t",
            time_samples[argmin(conflicts_samples)],"\t",
            nsamples/sum(time_samples)
        ) # solutions per second irrelevant, isn't it? (since there is no local search loop)
    end
end

INSTANCE NAME AND k	MIN CONFL	MEAN CONFL	MAX CONFL	TOTAL TIME	TIME BEST SOL	SOLS PER SECOND
flat300_26_0.col k=26	226	226.0	226	0.0020858	0.0002356	4794.323520951194
le450_15c.col k=15	362	362.0	362	0.0021066	0.00024400000000000002	4746.985664103294
dsjc125.1.col k=5	47	47.0	47	0.00010420000000000001	1.36e-5	95969.28982725527
dsjc125.9.col k=44	21	21.0	21	0.00012620000000000003	1.7500000000000002e-5	79239.30269413628
dsjc250.1.col k=8	88	88.0	88	0.0005214	5.19e-5	19179.133103183736
dsjc250.9.col k=72	61	61.0	61	0.0006502000000000001	6.510000000000001e-5	15379.88311288834
dsjc250.5.col k=28	85	85.0	85	0.0015033	0.000145	6652.032195835828


dsjc1000.5.col k=86	378	378.0	378	0.0255467	0.0026574000000000003	391.43999029228826


dsjc1000.5.col k=85	398	398.0	398	0.025178299999999997	0.0028216	397.16740208830623


dsjc1000.5.col k=84	412	412.0	412	0.023184100000000003	0.0024128	431.3300926065708


# 2) Structure de voisinage

On choisit d'utiliser un opérateur de voisinage simple:
 - Appliquer la couleur $i$ au nœud $j$ choisis de manière à minimiser le nombre de conflits.
   - On choisit un couple $(i,j)$ au hasard parmi tous les couples minimisant le nombre de conflits.
   - On tente également d'éviter que la nouvelle couleur soit la même que l'ancienne si possible.
   - Pour cela, on maintient une matrice des conflits stockant pour chaque nœud $j$ et chaque couleur $i$ le nombre de conflits en ce nœud $j$ si l'on lui applique la couleur $i$.

On obtient donc une structure de voisinage qui contient toutes les solutions dont un unique nœud diffère par sa couleur de la solution d'origine.

In [12]:
function fill_conflicts(instance::Instance,solution::Solution)
    nbr_collision(instance,solution)
    for i = 1:length(instance)
        for color = 1:instance.k
            solution.conflictmatrix[color,i] = nb_collision_sommet_couleur(instance,solution,i,color) 
        end
    end
end

function update_conflicts(instance::Instance,solution::Solution,node::Int,color::Int)
    for othernode = 1:length(instance)
        if instance.graphe[othernode,node]
            for othercolor in 1:instance.k
                solution.conflictmatrix[othercolor,othernode] += (othercolor==color) - (othercolor==solution.nodecolors[node])
            end
        end
    end
    solution.obj += solution.conflictmatrix[color,node] - solution.conflictmatrix[solution.nodecolors[node],node]
    solution.nodecolors[node] = color
end

"""Opérateur appliquant une couleur à un nœud de manière à minimiser les conflits."""
function simple_neighbor(instance::Instance,solution::Solution)
    if iszero(solution.conflictmatrix)
        fill_conflicts(instance,solution)
    end
    possible_changes = findall(
        solution.conflictmatrix .== minimum(solution.conflictmatrix) .&&
        1:instance.k .!= permutedims(solution.nodecolors)) # Éviter de garder la même solution si possible
    if isempty(possible_changes)
        return # minimum local
    end
    color,node = Tuple(rand(possible_changes)) # Modification aléatoire
    update_conflicts(instance,solution,node,color)
end

simple_neighbor

In [13]:
# Test de descente locale simple
begin
    instance = instance_list[3]
    solution = sol_alea(instance)

    for t = 1:25
        simple_neighbor(instance,solution)
        println("t: ",t,"\t","conflits: ",solution.obj)
    end
end

t: 1	conflits: 130
t: 2	conflits: 128
t: 3	conflits: 124
t: 4	conflits: 123
t: 5	conflits: 121
t: 6	conflits: 120
t: 7	conflits: 117
t: 8	conflits: 115
t: 9	conflits: 114
t: 10	conflits: 112
t: 11	conflits: 111
t: 12	conflits: 110
t: 13	conflits: 109
t: 14	conflits: 109
t: 15	conflits: 106
t: 16	conflits: 105
t: 17	conflits: 104
t: 18	conflits: 102
t: 19	conflits: 102
t: 20	conflits: 102
t: 21	conflits: 95
t: 22	conflits: 94
t: 23	conflits: 91
t: 24	conflits: 91
t: 25	conflits: 90


# 3) Métaheuristique proposée

On choisit

In [14]:
"""Version de `simple_neighbor` qui implémente le tabou."""
function tabu_neighbor(instance::Instance,solution::Solution,tabulength::Int,it::Int)
    if iszero(solution.conflictmatrix)
        fill_conflicts(instance,solution)
    end
    tabupenalizedconflictmatrix = solution.conflictmatrix + length(instance)^2*(
        solution.tabuexpiry .> it .|| # TABOU (application)
        1:instance.k .== permutedims(solution.nodecolors)) # Garder la même solution est interdit
    possible_changes = findall(solution.conflictmatrix .== minimum(tabupenalizedconflictmatrix))
    color,node = Tuple(rand(possible_changes)) # Modification aléatoire
    solution.tabuexpiry[color,node] = it+tabulength # TABOU (ajout)
    update_conflicts(instance,solution,node,color)
end

tabu_neighbor

In [19]:
# Test de descente locale avec tabou
begin
    instance = instance_list[3]
    solution = sol_alea(instance)

    for t = 1:500
        tabu_neighbor(instance,solution,10,t)
        println("t: ",t,"\t","conflits: ",solution.obj)
    end
end

t: 1	conflits: 147
t: 2	conflits: 146
t: 3	conflits: 145
t: 4	conflits: 142
t: 5	conflits: 142
t: 6	conflits: 142
t: 7	conflits: 137
t: 8	conflits: 133
t: 9	conflits: 131
t: 10	conflits: 124
t: 11	conflits: 124
t: 12	conflits: 119
t: 13	conflits: 115
t: 14	conflits: 115
t: 15	conflits: 115
t: 16	conflits: 112
t: 17	conflits: 111
t: 18	conflits: 109
t: 19	conflits: 109
t: 20	conflits: 105
t: 21	conflits: 105
t: 22	conflits: 105
t: 23	conflits: 105
t: 24	conflits: 104
t: 25	conflits: 104
t: 26	conflits: 104
t: 27	conflits: 104
t: 28	conflits: 102
t: 29	conflits: 102
t: 30	conflits: 102
t: 31	conflits: 100
t: 32	conflits: 100
t: 33	conflits: 100
t: 34	conflits: 98
t: 35	conflits: 98
t: 36	conflits: 98
t: 37	conflits: 98
t: 38	conflits: 98
t: 39	conflits: 98
t: 40	conflits: 97
t: 41	conflits: 97
t: 42	conflits: 95
t: 43	conflits: 95
t: 44	conflits: 95
t: 45	conflits: 95
t: 46	conflits: 93
t: 47	conflits: 92
t: 48	conflits: 92
t: 49	conflits: 92
t: 50	conflits: 92
t: 51	conflits: 92
t: 52	c

Algo génétique

In [16]:
# On commence par coder le tabou et si on a le temps on fait le génétique
# ça a l'air de prendre beaucoup plus de temps que je croyais

"""Renvoie une solution aléatoire selon la règle de la roue de la fortune."""
function roue_fortune(instance::Instance,population::Vector{Solution})
    # Solutions supposées évaluées
    weights = [1/sample.obj for sample ∈ population]
    cumw = cumsum(weights)
    r = rand(1:sum(weights))
    return findfirst(r .<= cumw)
end

function genetique(μ::Int)
    
end

genetique (generic function with 1 method)